# Introduction

All of the maps you will create in this course portray the surface of the earth in two dimensions.  But, as you know, the world is actually a three-dimensional globe, and so we have to use a special method, called a **map projection**, to render it as a flat surface.  In general, no map projection can ever be 100% accurate.  Each projection distorts the surface of the Earth in some way, while retaining some useful property.  For instance,
- the *equal-area* projections (like "Lambert Cylindrical Equal Area", or "Africa Albers Equal Area Conic") preserve area.  This is a good choice, if you'd like to calculate the area of a country or city.
- the *equidistant* projections (like "Azimuthal Equidistant projection") preserve distance.  This would be a good choice for calculating flight distance.

We use a **coordinate reference system (CRS)** to show how the projected points correspond to real locations on Earth.  In this tutorial, you'll learn more about coordinate reference systems, along with how to use them in GeoPandas.

In [ ]:
#$HIDE_INPUT$
import geopandas as gpd
import pandas as pd

# Setting the CRS

We can identify common coordinate reference systems according to their [European Petroleum Survey Group (EPSG)](http://www.epsg.org/) codes.  

For instance, in the code cell below, the CRS of the GeoDataFrame `regions` is [EPSG 32630](https://epsg.io/32630).  This corresponds to a "Mercator" projection that preserves angles (making it incredibly useful for sea navigation, for instance) and slightly distorts area.

As you'll notice, when we create a GeoDataFrame _from a shapefile_, the CRS is already imported for us (and it was saved as part of the shapefile).  

In [ ]:
# Load a GeoDataFrame containing regions in Ghana
regions = gpd.read_file("../input/geospatial-learn-course-data/ghana/ghana/Regions/Map_of_Regions_in_Ghana.shp")
print(regions.crs)

However, when creating a GeoDataFrame _from a CSV file_, we have to set the CRS.  [EPSG 4326](https://epsg.io/4326) corresponds to coordinates in latitude and longitude.

In [ ]:
# Create a DataFrame with health facilities in Ghana
facilities_df = pd.read_csv("../input/geospatial-learn-course-data/ghana/ghana/health_facilities.csv")

# Convert the DataFrame to a GeoDataFrame
facilities = gpd.GeoDataFrame(facilities_df, geometry=gpd.points_from_xy(facilities_df.Longitude, facilities_df.Latitude))

# Set the coordinate reference system (CRS) to EPSG 4326
facilities.crs = {'init': 'epsg:4326'}

# View the first five rows of the GeoDataFrame
facilities.head()

In the code cell above, to create a GeoDataFrame from a CSV file, we needed to use both Pandas and GeoPandas:
- We begin by creating a DataFrame containing columns with latitude and longitude coordinates.
- To convert it to a GeoDataFrame, we use `gpd.GeoDataFrame()`.  
- The `gpd.points_from_xy()` function creates `Point` objects from the latitude and longitude columns.

# Re-projecting

Re-projecting refers to the process of changing the CRS.  This is done in GeoPandas with the `to_crs()` method.

When plotting multiple GeoDataFrames, it's important that they all use the same CRS.  In the code cell below, we change the CRS of the `facilities` GeoDataFrame to match the CRS of `regions` before plotting it.

In [ ]:
# Create a map
ax = regions.plot(figsize=(8,8), color='whitesmoke', linestyle=':', edgecolor='black')
facilities.to_crs(epsg=32630).plot(markersize=1, ax=ax)

The `to_crs()` method modifies only the "geometry" column: all other columns are left as-is.

In [ ]:
# The "Latitude" and "Longitude" columns are unchanged
facilities.to_crs(epsg=32630).head()

In case the EPSG code is not available in GeoPandas, we can change the CRS with what's known as the "proj4 string" of the CRS.  For instance, the "Africa Albers Equal Area Conic" projection corresponds to [EPSG 102022](https://epsg.io/102022) and has this proj4 string:
```
+proj=aea +lat_1=20 +lat_2=-23 +lat_0=0 +lon_0=25 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs
```

which you can find on [this website](https://epsg.io/102022), under "Description".

In [ ]:
# Change the CRS to EPSG 102022
facilities.to_crs("+proj=aea +lat_1=20 +lat_2=-23 +lat_0=0 +lon_0=25 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs").head()

# Attributes of geometric objects

As you learned in the first tutorial, for an arbitrary GeoDataFrame, the type in the "geometry" column depends on what we are trying to show: for instance, we might use:
- a `Point` for the epicenter of an earthquake, 
- a `Line` for a street, or 
- a `Polygon` to show country boundaries.

All three types of geometric objects have built-in attributes that you can use to quickly analyze the dataset.  For instance, you can get the x- and y-coordinates of a `Point` from the `x` and `y` attributes, respectively.

In [ ]:
# Get the x-coordinate of each point
facilities.geometry.x.head()

And, you can get the length of a `LineString` from the `length` attribute.  

Or, you can get the area of a `Polygon` from the `area` attribute.

In [ ]:
# Calculate the area (in square meters) of each polygon in the GeoDataFrame 
regions.loc[:, "AREA"] = regions.geometry.to_crs("+proj=aea +lat_1=20 +lat_2=-23 +lat_0=0 +lon_0=25 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs").area / 10**6

print("Area of Ghana: {} square kilometers".format(regions.AREA.sum()))
regions.head()

In the code cell above, we re-project to "Africa Albers Equal Area Conic" (EPSG 102022) before calculating the area, to ensure better accuracy.  (_This is the same re-projection we did in the previous section!_)  This yields the total area of Ghana as approximately 239473 square kilometers.  

In this case, using the original "Mercator" projection yields a slightly less accuate answer.

In [ ]:
print("CRS:", regions.crs)
print("(Slightly less accurate) calculated area of Ghana: {} square kilometers".format(regions.geometry.area.sum() / 10**6))

# Your turn

Use what you've learned to **[track bird migration to South America](#$NEXT_NOTEBOOK_URL$)**.